In [10]:
# ! pip install convokit

In [9]:
# ! pip install seaborn

In [8]:
# ! pip install nltk

In [7]:
# ! pip install scikit-learn

In [33]:
# ! pip install genism

In [32]:
# ! pip install scipy

In [5]:
from convokit import Corpus, download
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [31]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

ImportError: cannot import name 'triu' from 'scipy.linalg' (C:\Users\joda\AppData\Roaming\Python\Python311\site-packages\scipy\linalg\__init__.py)

In [11]:
# nltk.download('all')

In [23]:
seed = 38

In [12]:
corpus = Corpus(filename=download("movie-corpus"))

In [13]:
utterances = corpus.get_utterances_dataframe()

In [14]:
conversations = corpus.get_conversations_dataframe()

In [15]:
def extract_first_genre(meta_genre):
    if isinstance(meta_genre, str) and meta_genre.startswith('[') and meta_genre.endswith(']'):
        genres = meta_genre.strip("[]").replace("'", "").split(", ") 
        if genres:
            return genres[0]
    return meta_genre

conversations['meta.genre'] = conversations['meta.genre'].apply(extract_first_genre)

In [16]:
merged_df = pd.merge(utterances, conversations, left_on='conversation_id', right_on='id', suffixes=('_utterance', '_conversation'))


In [17]:
merged_df['meta.rating'] = pd.to_numeric(merged_df['meta.rating'], errors='coerce')
merged_df['meta.votes'] = pd.to_numeric(merged_df['meta.votes'], errors='coerce')

In [21]:
merged_df = merged_df.dropna(subset=['meta.genre'])

merged_df = merged_df[merged_df['meta.genre'] != '']

In [24]:
merged_df = merged_df.sample(frac=1, random_state=seed).reset_index(drop=True)

In [25]:
s = stopwords.words('english')
s.extend([',' , '.' , '?' , ',' , ',' ,':' ,';',"'", '“', '”', '!', '’','...', '....','--', '_'])

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [word for word in text if word not in s and not any(char.isdigit() for char in word)]
    return ' '.join(text)

In [26]:
merged_df['Processed_Text'] = merged_df['text'].apply(preprocess_text)

In [ ]:
tokenized_text = [word_tokenize(sentence.lower()) for sentence in merged_df['Processed_Text']]
model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
X_embeddings = model.wv['sentence']
print(X_embeddings)


In [ ]:
vectorizer = TfidfVectorizer()
X_tfid = vectorizer.fit_transform(merged_df['Processed_Text'])
print(X_tfid)

In [27]:
y= merged_df['meta.genre']